<a href="https://colab.research.google.com/github/Bongfire/full-stack-professional/blob/main/CapsNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 12.7 MB/s eta 0:00:00


In [ ]:
#Cell 1
# Import necessary libraries
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, average_precision_score
import optuna
import time
import matplotlib.pyplot as plt

In [ ]:
# Define paths and hyperparameters
#Cell 2
dataset_path = '/content/drive/MyDrive/Original Dataset'
input_shape = (224, 224, 3)
n_classes = 5  # Update this based on your dataset's number of classes

In [ ]:
# Preprocessing functions
#Cell 3

def resize_images(image, size=(224, 224)):
    return cv2.resize(image, size)

def normalize_images(image):
    normalized = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
    return normalized.astype(np.uint8)

def detect_contours(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours[0] if contours else None

def create_binary_mask(image, contour):
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)
    return cv2.bitwise_and(image, image, mask=mask)

def calculate_hu_moments(contour):
    moments = cv2.moments(contour)
    return cv2.HuMoments(moments).flatten()

def preprocess_image(image):
    resized_image = resize_images(image)
    normalized_image = normalize_images(resized_image)
    contour = detect_contours(normalized_image)
    if contour is not None:
        cropped_image = create_binary_mask(resized_image, contour)
        hu_features = calculate_hu_moments(contour)
        return cropped_image, hu_features
    return resized_image, None

In [ ]:
# Load and preprocess dataset
#Cell 4
processed_images = []
hu_features_dataset = []
labels = []

for idx, disease_folder in enumerate(os.listdir(dataset_path)):
    disease_path = os.path.join(dataset_path, disease_folder)
    if os.path.isdir(disease_path):
        for image_file in os.listdir(disease_path):
            image_path = os.path.join(disease_path, image_file)
            image = cv2.imread(image_path)
            if image is not None:
                cropped_image, hu_features = preprocess_image(image)
                if cropped_image is not None and hu_features is not None:
                    processed_images.append(cropped_image)
                    hu_features_dataset.append(hu_features)
                    labels.append(idx)

# Convert to numpy arrays and one-hot encode labels
processed_images = np.array(processed_images)
hu_features_dataset = np.array(hu_features_dataset)
labels = np.array(labels)
y_train_one_hot = tf.keras.utils.to_categorical(labels, num_classes=n_classes)

# Split dataset
X_train_images, X_test_images, X_train_hu, X_test_hu, y_train, y_test = train_test_split(
    processed_images, hu_features_dataset, y_train_one_hot, test_size=0.2, random_state=42)

In [ ]:
# CapsNet architecture
#Cell 5
def CapsNet(input_shape, n_classes, n_routing=3):
    image_input = layers.Input(shape=input_shape, name="Image_Input")
    hu_input = layers.Input(shape=(7,), name="Hu_Moments_Input")

    # Convolutional layers
    conv1 = layers.Conv2D(256, (9, 9), activation='relu', padding='same', name="Conv1")(image_input)
    primary_caps = layers.Conv2D(32 * 8, (9, 9), strides=2, activation='relu', padding='same', name="Primary_Caps")(conv1)
    primary_caps = layers.Reshape([-1, 8], name="Reshape_Primary")(primary_caps)

    # Digit Capsule layer with dynamic routing
    digit_caps = layers.Dense(n_classes, activation='softmax', name="Digit_Caps")(primary_caps)

    # Flatten and combine with Hu moments
    digit_caps_flat = layers.Flatten(name="Flatten_Caps")(digit_caps)
    combined_features = layers.Concatenate(name="Combine_Features")([digit_caps_flat, hu_input])

    # Output layer
    output = layers.Dense(n_classes, activation='softmax', name="Output_Layer")(combined_features)

    model = models.Model(inputs=[image_input, hu_input], outputs=output, name="CapsNet_with_Hu")
    return model

# Margin loss function
def margin_loss(y_true, y_pred):
    m_plus = 0.9
    m_minus = 0.1
    lambda_val = 0.5
    L_k = y_true * tf.square(tf.maximum(0., m_plus - y_pred)) + \
          lambda_val * (1 - y_true) * tf.square(tf.maximum(0., y_pred - m_minus))
    return tf.reduce_mean(tf.reduce_sum(L_k, axis=1))

In [ ]:
# Hyperparameter optimization with Optuna
#Cell 6
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    n_routing = trial.suggest_int("n_routing", 2, 5)

    # Build and compile CapsNet model with hyperparameters
    capsnet_model = CapsNet(input_shape, n_classes, n_routing)
    capsnet_model.compile(optimizer=optimizers.Adam(learning_rate),
                          loss=margin_loss, metrics=['accuracy'])

    history = capsnet_model.fit(
        [X_train_images, X_train_hu], y_train,
        batch_size=batch_size,
        epochs=10,  # Fewer epochs for hyperparameter search
        validation_split=0.2,
        verbose=0
    )
    return history.history['val_accuracy'][-1]

# Run optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)
best_params = study.best_params
print("Best Hyperparameters:", best_params)

[I 2025-04-07 05:43:10,342] A new study created in memory with name: no-name-1b3d427f-6a49-4b27-b96f-8ea614fae9f4


In [ ]:
# Train final model with best hyperparameters
#Cell 7
final_model = CapsNet(input_shape, n_classes, best_params["n_routing"])
final_model.compile(
    optimizer=optimizers.Adam(best_params["learning_rate"]),
    loss=margin_loss,
    metrics=['accuracy']
)

# Add early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Train the model
history = final_model.fit(
    [X_train_images, X_train_hu], y_train,
    batch_size=best_params["batch_size"],
    epochs=100,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
# Evaluate the model
#Cell 8
y_pred = final_model.predict([X_test_images, X_test_hu])
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Print evaluation metrics
print("\nClassification Report:")
print(classification_report(y_test_classes, y_pred_classes))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_classes, y_pred_classes))
print("\nF1-Score:", f1_score(y_test_classes, y_pred_classes, average='weighted'))
print("mAP:", average_precision_score(y_test, y_pred, average='macro'))

# Calculate and print FPS
start_time = time.time()
_ = final_model.predict([X_test_images, X_test_hu])
fps = len(X_test_images) / (time.time() - start_time)
print("\nFrames Per Second (FPS):", fps)

# Print final accuracy
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]
test_acc = np.mean(y_pred_classes == y_test_classes)
print("\nFinal Training Accuracy:", train_acc)
print("Final Validation Accuracy:", val_acc)
print("Final Test Accuracy:", test_acc)

In [ ]:
# Plot training history
#Cell 9
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
# Save the trained model
#Cell 10
final_model.save("optimized_capsnet_model.h5")

# Convert to TensorFlow Lite for mobile deployment
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
tflite_model = converter.convert()
with open("capsnet_model.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
# Frames per second (FPS)
# Frames per second (FPS)
# Frames per second (FPS)
# Frames per second (FPS)
#Cell 11
import time # Import the 'time' module to access its functions

start_time = time.time()
_ = model.predict([X_test_images, X_test_hu]) # Replace 'capsnet_model' with 'model'
fps = len(X_test_images) / (time.time() - start_time)
print("Frames Per Second (FPS):", fps)